In [1]:
from pyspark.sql.functions import *
from  pyspark.sql.functions import abs
import time
import pyspark.sql.functions as F
from pyspark.sql.types import *

In [2]:
df = spark.read.csv("hdfs://orion11:11101/3hr_sample/*", sep=r'\t', header=True, inferSchema="true")

In [3]:
df.limit(5).toPandas()

,1_time,2_lat,3_lon,albedo_surface,precipitable_water_entire_atmosphere_single_layer,pressure_maximum_wind,pressure_surface,pressure_tropopause,relative_humidity_zerodegc_isotherm,snow_depth_surface,temperature_surface,temperature_tropopause,total_cloud_cover_entire_atmosphere_single_layer,total_precipitation_surface_3_hour_accumulation,vegetation_surface,visibility_surface,wilting_point_surface,wind_speed_gust_surface
0,1455440400000,28.862712,-80.155704,6.0,null,17126.896,102390.0,23103.373,35.0,0.0,295.9439,219.42467,null,0.0,0.0,24223.668,0.0,null
1,1455440400000,57.697196,-79.334581,65.0,null,11126.896,101925.0,30303.373,76.0,0.049999997,243.69392,213.04967,null,0.0,0.0,24023.668,0.0,null
2,1455440400000,36.639956,-120.499569,16.0,null,19326.896,100150.0,19703.373,26.0,0.0,282.5689,210.17467,null,0.0,20.5,24223.668,0.1025,null
3,1455440400000,33.129387,-82.595627,15.0,null,14526.896,101385.0,20703.373,52.0,0.0,272.4439,211.42467,null,0.0,41.0,24223.668,0.0275,null
4,1455440400000,47.168122,-119.719137,16.25,null,25126.896,97359.0,21103.373,70.0,0.0,272.8189,214.67467,null,0.0,6.0,23.668581,0.022499999,null


In [47]:
from scipy.stats import pearsonr
x = [0, 0, 1, 1, 1]
y = [0, 3, 5 ,7 ,4]

correlation, p_value = pearsonr(x, y)
# Calculate correlation and p-value


print(correlation)

0.8111474582373881


In [6]:
# (http://tools.ietf.org/html/rfc4648)
__base32 = '0123456789bcdefghjkmnpqrstuvwxyz'

def encode(latitude, longitude, precision=12):
    """
    Encode a position given in float arguments latitude, longitude to
    a geohash which will have the character count precision.
    """
    lat_interval = (-90.0, 90.0)
    lon_interval = (-180.0, 180.0)
    geohash = []
    bits = [16, 8, 4, 2, 1]
    bit = 0
    ch = 0
    even = True
    while len(geohash) < precision:
        if even:
            mid = (lon_interval[0] + lon_interval[1]) / 2
            if longitude > mid:
                ch |= bits[bit]
                lon_interval = (mid, lon_interval[1])
            else:
                lon_interval = (lon_interval[0], mid)
        else:
            mid = (lat_interval[0] + lat_interval[1]) / 2
            if latitude > mid:
                ch |= bits[bit]
                lat_interval = (mid, lat_interval[1])
            else:
                lat_interval = (lat_interval[0], mid)
        even = not even
        if bit < 4:
            bit += 1
        else:
            geohash += __base32[ch]
            bit = 0
            ch = 0
    return ''.join(geohash)

In [7]:
def convertToGeoHash(lat, lon):
    return encode(lat, lon,2)

udfConvertToGeoHash = F.udf(convertToGeoHash, StringType())

geohashDf = df.withColumn("5_hash", udfConvertToGeoHash("2_lat", "3_lon"))

geohashDf.limit(5).toPandas()

,1_time,2_lat,3_lon,albedo_surface,precipitable_water_entire_atmosphere_single_layer,pressure_maximum_wind,pressure_surface,pressure_tropopause,relative_humidity_zerodegc_isotherm,snow_depth_surface,temperature_surface,temperature_tropopause,total_cloud_cover_entire_atmosphere_single_layer,total_precipitation_surface_3_hour_accumulation,vegetation_surface,visibility_surface,wilting_point_surface,wind_speed_gust_surface,5_hash
0,1455440400000,28.862712,-80.155704,6.0,null,17126.896,102390.0,23103.373,35.0,0.0,295.9439,219.42467,null,0.0,0.0,24223.668,0.0,null,dj
1,1455440400000,57.697196,-79.334581,65.0,null,11126.896,101925.0,30303.373,76.0,0.049999997,243.69392,213.04967,null,0.0,0.0,24023.668,0.0,null,f4
2,1455440400000,36.639956,-120.499569,16.0,null,19326.896,100150.0,19703.373,26.0,0.0,282.5689,210.17467,null,0.0,20.5,24223.668,0.1025,null,9q
3,1455440400000,33.129387,-82.595627,15.0,null,14526.896,101385.0,20703.373,52.0,0.0,272.4439,211.42467,null,0.0,41.0,24223.668,0.0275,null,dj
4,1455440400000,47.168122,-119.719137,16.25,null,25126.896,97359.0,21103.373,70.0,0.0,272.8189,214.67467,null,0.0,6.0,23.668581,0.022499999,null,c2


In [8]:
def convertToMonth(value):
    return time.strftime('%Y', time.localtime(int(value)/1000))

udfConvertToMonth = F.udf(convertToMonth, StringType())

monthDf = geohashDf.withColumn("1_time", udfConvertToMonth("1_time"))
monthDf.limit(5).toPandas()

,1_time,2_lat,3_lon,albedo_surface,precipitable_water_entire_atmosphere_single_layer,pressure_maximum_wind,pressure_surface,pressure_tropopause,relative_humidity_zerodegc_isotherm,snow_depth_surface,temperature_surface,temperature_tropopause,total_cloud_cover_entire_atmosphere_single_layer,total_precipitation_surface_3_hour_accumulation,vegetation_surface,visibility_surface,wilting_point_surface,wind_speed_gust_surface,5_hash
0,2016,28.862712,-80.155704,6.0,null,17126.896,102390.0,23103.373,35.0,0.0,295.9439,219.42467,null,0.0,0.0,24223.668,0.0,null,dj
1,2016,57.697196,-79.334581,65.0,null,11126.896,101925.0,30303.373,76.0,0.049999997,243.69392,213.04967,null,0.0,0.0,24023.668,0.0,null,f4
2,2016,36.639956,-120.499569,16.0,null,19326.896,100150.0,19703.373,26.0,0.0,282.5689,210.17467,null,0.0,20.5,24223.668,0.1025,null,9q
3,2016,33.129387,-82.595627,15.0,null,14526.896,101385.0,20703.373,52.0,0.0,272.4439,211.42467,null,0.0,41.0,24223.668,0.0275,null,dj
4,2016,47.168122,-119.719137,16.25,null,25126.896,97359.0,21103.373,70.0,0.0,272.8189,214.67467,null,0.0,6.0,23.668581,0.022499999,null,c2


In [9]:
groupedHumTempDf = monthDf.groupBy("5_hash","1_time").agg(F.avg("relative_humidity_zerodegc_isotherm").alias("relative_humidity_zerodegc_isotherm"),F.max("temperature_surface").alias("temperature_surface"))

In [11]:
groupedHumTempDf.limit(5).toPandas()

,5_hash,1_time,relative_humidity_zerodegc_isotherm,temperature_surface
0,9m,2015,25.532951,328.76392
1,8y,2019,44.216567,298.83997
2,dn,2014,63.183983,276.4975
3,dn,2015,50.701690,319.364
4,c8,2017,67.048192,null


In [12]:
groupedDf = groupedHumTempDf.drop("relative_humidity_zerodegc_isotherm")

In [13]:
groupedDf.limit(5).toPandas()

,5_hash,1_time,temperature_surface
0,9m,2015,328.76392
1,8y,2019,298.83997
2,dn,2014,276.4975
3,dn,2015,319.364
4,c8,2017,null


In [9]:
# With pandas
# a = groupedDf.toPandas()
# wide_fmt = a.pivot(index='5_hash', columns='1_time', values='temperature_surface')
# wide_fmt
# wide_fmt.info()
# Converting pandas df to spark df
# wideFmtSparkDf = spark.createDataFrame(wide_fmt)

In [14]:
# With spark
wide_fmt = monthDf.groupBy("5_hash").pivot("1_time").agg(F.max("temperature_surface").alias("temperature_surface"))

In [15]:
wide_fmt.limit(5).toPandas()

,5_hash,2014,2015,2016,2017,2018,2019
0,f2,273.6225,310.83887,null,null,312.14615,310.55997
1,c0,285.7475,317.34937,null,null,311.72736,309.67
2,f6,254.6225,305.88745,null,null,304.06995,306.63
3,cc,267.8725,312.84448,null,null,317.43997,315.60962
4,bc,281.9975,290.5183,null,null,290.585,291.3078


In [17]:
def monotonicallyIncreasing(y1,y2,y3,y4,y5,y6):
    test_list = []
    if y1 != "null":
        test_list.append(y1)
    if y2 != "null":
        test_list.append(y2)
    if y3 != "null":
        test_list.append(y3)
    if y4 != "null":
        test_list.append(y4)
    if y5 != "null":
        test_list.append(y5)
    if y6 != "null":
        test_list.append(y6)
    return all(i < j for i, j in zip(test_list, test_list[1:])) 
#     return test_list
        

udfMonotonicallyIncreasing = F.udf(monotonicallyIncreasing, StringType())
is_increasing_Df = wide_fmt.withColumn("is_increasing", udfMonotonicallyIncreasing("2014","2015","2016","2017","2018","2019"))

In [18]:
is_increasing_Df.limit(11).toPandas()

,5_hash,2014,2015,2016,2017,2018,2019,is_increasing
0,f2,273.6225,310.83887,null,null,312.14615,310.55997,false
1,c0,285.7475,317.34937,null,null,311.72736,309.67,false
2,f6,254.6225,305.88745,null,null,304.06995,306.63,false
3,cc,267.8725,312.84448,null,null,317.43997,315.60962,false
4,bc,281.9975,290.5183,null,null,290.585,291.3078,true
5,f4,256.8725,304.8037,null,null,309.94998,304.54517,false
6,b9,281.7475,288.48218,null,null,287.78302,289.90732,false
7,9z,265.8725,320.08936,null,null,319.8566,318.53,false
8,9u,297.9975,323.3811,null,null,325.96997,326.4664,true
9,f8,281.4975,312.3003,null,null,311.11307,311.12177,false


In [19]:
increasingTempGeoHashes = is_increasing_Df.filter(col("is_increasing") ==  True)

In [20]:
increasingTempGeoHashes.limit(4).toPandas()

,5_hash,2014,2015,2016,2017,2018,2019,is_increasing
0,bc,281.9975,290.5183,null,null,290.585,291.3078,true
1,9u,297.9975,323.3811,null,null,325.96997,326.4664,true
2,9x,274.3725,319.81274,null,null,321.16968,321.41,true
3,9s,299.1225,328.09082,null,null,331.566,331.61,true
4,9e,301.9975,326.83325,null,null,328.0671,328.85913,true
5,9p,288.6225,318.38745,null,null,318.41995,318.43,true
6,dp,278.2475,312.4287,null,null,314.58997,316.2473,true
7,bf,281.8725,304.7229,null,null,305.4071,308.69943,true
8,f0,277.3725,310.5537,null,null,312.83,314.73734,true
9,8y,290.9975,296.68628,null,null,296.84616,298.83997,true


In [21]:
increaseTempGeoHashList = list(increasingTempGeoHashes.select('5_hash').toPandas()['5_hash'])

In [22]:
increaseTempGeoHashList

['bc', '9u', '9x', '9s', '9e', '9p', 'dp', 'bf', 'f0', '8y']

In [25]:
# groupedHumDf.filter(col['5_hash'].isin(increaseTempGeoHashList)).show()
corrDf = groupedHumTempDf.filter(col("5_hash").isin(increaseTempGeoHashList)).select("5_hash","1_time","relative_humidity_zerodegc_isotherm","temperature_surface")

In [26]:
corrDf.limit(11).toPandas()

,5_hash,1_time,relative_humidity_zerodegc_isotherm,temperature_surface
0,8y,2019,44.216567,298.83997
1,9e,2015,50.282616,326.83325
2,bf,2015,77.759759,304.7229
3,9u,2015,41.340677,323.3811
4,9u,2017,35.508914,null
5,8y,2017,47.083304,null
6,9e,2014,21.146939,301.9975
7,9s,2017,39.180013,null
8,9s,2018,43.306710,331.566
9,bc,2014,74.665127,281.9975


In [28]:
orderedCorrDF = corrDf.orderBy("5_hash","1_time")

In [29]:
orderedCorrDF.show()

+------+------+-----------------------------------+-------------------+
|5_hash|1_time|relative_humidity_zerodegc_isotherm|temperature_surface|
+------+------+-----------------------------------+-------------------+
|    8y|  2014|                 11.212418300653594|           290.9975|
|    8y|  2015|                  39.90850851132732|          296.68628|
|    8y|  2016|                  46.19603099111853|               null|
|    8y|  2017|                  47.08330374074477|               null|
|    8y|  2018|                  41.22466285850391|          296.84616|
|    8y|  2019|                  44.21656745535491|          298.83997|
|    9e|  2014|                 21.146938775510204|           301.9975|
|    9e|  2015|                 50.282615732166754|          326.83325|
|    9e|  2016|                  43.62167813002086|               null|
|    9e|  2017|                 39.194828838286156|               null|
|    9e|  2018|                  45.41812503872096|           32

In [36]:
a = corrDf.toPandas()
wide_fmt1 = a.pivot(index='5_hash', columns='1_time', values=['5_hash','temperature_surface','relative_humidity_zerodegc_isotherm'])
wide_fmt1

5_hash                          temperature_surface                   \
1_time   2014 2015 2016 2017 2018 2019                2014       2015  2016   
5_hash                                                                        
8y         8y   8y   8y   8y   8y   8y            290.9975  296.68628  null   
9e         9e   9e   9e   9e   9e   9e            301.9975  326.83325  null   
9p         9p   9p   9p   9p   9p   9p            288.6225  318.38745  null   
9s         9s   9s   9s   9s   9s   9s            299.1225  328.09082  null   
9u         9u   9u   9u   9u   9u   9u            297.9975   323.3811  null   
9x         9x   9x   9x   9x   9x   9x            274.3725  319.81274  null   
bc         bc   bc   bc   bc   bc   bc            281.9975   290.5183  null   
bf         bf   bf   bf   bf   bf   bf            281.8725   304.7229  null   
dp         dp   dp   dp   dp   dp   dp            278.2475   312.4287  null   
f0         f0   f0   f0   f0   f0   f0            277.3725   310.5537  null   

                                   relative_humidity_zerodegc_isotherm  \
1_time  2017       2018       2019                                2014   
5_hash                                                                   
8y      null  296.84616  298.83997                             11.2124   
9e      null   328.0671  328.85913                             21.1469   
9p      null  318.41995     318.43                             26.5337   
9s      null    331.566     331.61                             60.3035   
9u      null  325.96997   326.4664                             26.8647   
9x      null  321.16968     321.41                             72.3879   
bc      null    290.585   291.3078                             74.6651   
bf      null   305.4071  308.69943                             97.2561   
dp      null  314.58997   316.2473                             70.7423   
f0      null     312.83  314.73734                             80.3593   

                                                     
1_time     2015     2016     2017     2018     2019  
5_hash                                               
8y      39.9085   46.196  47.0833  41.2247  44.2166  
9e      50.2826  43.6217  39.1948  45.4181  45.2645  
9p       45.974   52.337  53.0908  47.4255  51.2625  
9s      46.4219  41.2114    39.18  43.3067  40.7718  
9u      41.3407  39.6205  35.5089  42.0777  38.9404  
9x      64.4904  64.5654  65.0879  63.9213  67.8083  
bc      74.7272  76.2708  76.3131  72.2432  71.4528  
bf      77.7598  79.0369  78.2417  76.3798  75.9711  
dp      58.4326  59.3929  59.3679  61.3979  61.0161  
f0      66.5443  67.0962  68.7725  65.4849  65.9502

In [37]:
wideFmt1SparkDf = spark.createDataFrame(wide_fmt1)

In [38]:
wideFmt1SparkDf.limit(5).toPandas()

,"('5_hash', '2014')","('5_hash', '2015')","('5_hash', '2016')","('5_hash', '2017')","('5_hash', '2018')","('5_hash', '2019')","('temperature_surface', '2014')","('temperature_surface', '2015')","('temperature_surface', '2016')","('temperature_surface', '2017')","('temperature_surface', '2018')","('temperature_surface', '2019')","('relative_humidity_zerodegc_isotherm', '2014')","('relative_humidity_zerodegc_isotherm', '2015')","('relative_humidity_zerodegc_isotherm', '2016')","('relative_humidity_zerodegc_isotherm', '2017')","('relative_humidity_zerodegc_isotherm', '2018')","('relative_humidity_zerodegc_isotherm', '2019')"
0,8y,8y,8y,8y,8y,8y,290.9975,296.68628,null,null,296.84616,298.83997,11.212418,39.908509,46.196031,47.083304,41.224663,44.216567
1,9e,9e,9e,9e,9e,9e,301.9975,326.83325,null,null,328.0671,328.85913,21.146939,50.282616,43.621678,39.194829,45.418125,45.264487
2,9p,9p,9p,9p,9p,9p,288.6225,318.38745,null,null,318.41995,318.43,26.533654,45.973955,52.337006,53.090753,47.425549,51.262530
3,9s,9s,9s,9s,9s,9s,299.1225,328.09082,null,null,331.566,331.61,60.303493,46.421898,41.211406,39.180013,43.306710,40.771838
4,9u,9u,9u,9u,9u,9u,297.9975,323.3811,null,null,325.96997,326.4664,26.864706,41.340677,39.620524,35.508914,42.077667,38.940404


In [39]:
tempHumList = wide_fmt1.values.tolist()

In [40]:
tempHumList

[['8y',
  '8y',
  '8y',
  '8y',
  '8y',
  '8y',
  '290.9975',
  '296.68628',
  'null',
  'null',
  '296.84616',
  '298.83997',
  11.212418300653594,
  39.90850851132732,
  46.19603099111853,
  47.08330374074477,
  41.22466285850391,
  44.21656745535491],
 ['9e',
  '9e',
  '9e',
  '9e',
  '9e',
  '9e',
  '301.9975',
  '326.83325',
  'null',
  'null',
  '328.0671',
  '328.85913',
  21.146938775510204,
  50.282615732166754,
  43.62167813002086,
  39.194828838286156,
  45.41812503872096,
  45.26448653452156],
 ['9p',
  '9p',
  '9p',
  '9p',
  '9p',
  '9p',
  '288.6225',
  '318.38745',
  'null',
  'null',
  '318.41995',
  '318.43',
  26.533653846153847,
  45.97395504393614,
  52.33700627979286,
  53.090753272084854,
  47.42554941165325,
  51.26252992125852],
 ['9s',
  '9s',
  '9s',
  '9s',
  '9s',
  '9s',
  '299.1225',
  '328.09082',
  'null',
  'null',
  '331.566',
  '331.61',
  60.30349344978166,
  46.42189844274342,
  41.21140588044996,
  39.180013297327065,
  43.30671013406952,
  40.771

In [43]:
for i in tempHumList:
    geohash = i[0]
    temp = i[6:12]
    hum = i[12:]
    print(temp, hum)

['290.9975', '296.68628', 'null', 'null', '296.84616', '298.83997'] [11.212418300653594, 39.90850851132732, 46.19603099111853, 47.08330374074477, 41.22466285850391, 44.21656745535491]
['301.9975', '326.83325', 'null', 'null', '328.0671', '328.85913'] [21.146938775510204, 50.282615732166754, 43.62167813002086, 39.194828838286156, 45.41812503872096, 45.26448653452156]
['288.6225', '318.38745', 'null', 'null', '318.41995', '318.43'] [26.533653846153847, 45.97395504393614, 52.33700627979286, 53.090753272084854, 47.42554941165325, 51.26252992125852]
['299.1225', '328.09082', 'null', 'null', '331.566', '331.61'] [60.30349344978166, 46.42189844274342, 41.21140588044996, 39.180013297327065, 43.30671013406952, 40.77183769582798]
['297.9975', '323.3811', 'null', 'null', '325.96997', '326.4664'] [26.86470588235294, 41.340676817452724, 39.62052433425469, 35.50891350265012, 42.07766733175509, 38.94040443527248]
['274.3725', '319.81274', 'null', 'null', '321.16968', '321.41'] [72.38785046728972, 64.

In [61]:
from scipy.stats import pearsonr
res = []
for i in tempHumList:
    geohash = i[0]
    temp = [0 if x == "null" else float(x) for x in i[6:12]]
    humidity = [0 if x == "null" else float(x) for x in i[12:]]
    correlation, p_value = pearsonr(temp, humidity)
    res.append((geohash,str(correlation)))
    

In [62]:
res

[('8y', '-0.46113040974179764'),
 ('9e', '0.014012919025987024'),
 ('9p', '-0.4509179772252986'),
 ('9s', '0.4328976712939623'),
 ('9u', '0.03709686305438359'),
 ('9x', '0.2750060043533481'),
 ('bc', '-0.7771805988932008'),
 ('bf', '0.14249577226255644'),
 ('dp', '0.31984315570757504'),
 ('f0', '0.06425264460115487')]

In [64]:
spark.createDataFrame(res, ['geohash', 'temp_humidity_correlation']).show()

+-------+-------------------------+
|geohash|temp_humidity_correlation|
+-------+-------------------------+
|     8y|     -0.46113040974179764|
|     9e|     0.014012919025987024|
|     9p|      -0.4509179772252986|
|     9s|       0.4328976712939623|
|     9u|      0.03709686305438359|
|     9x|       0.2750060043533481|
|     bc|      -0.7771805988932008|
|     bf|      0.14249577226255644|
|     dp|      0.31984315570757504|
|     f0|      0.06425264460115487|
+-------+-------------------------+

